# Pull Polygons from TerraMatch API for PPC (Simple)

This notebook sets up the process to pull polygon geometries and metadata from the TerraMatch API specifically for PPC projects (includes the hectares by
WWF ecoregion indicator).

Updated to include indicators and simplify preprocessing

In [1]:
import yaml
import pandas as pd
from tm_api_utils import pull_tm_api_data, patch_tm_api_data
from tqdm import tqdm
import json
import sys
from datetime import datetime
sys.path.append('../src/')
import api_utils as api
import process_tm_api_results as clean

## Set file paths

In [2]:
## PARAMS
# Naming convention
run_dir = 'ppc_tree_count_elig'
run_name = 'ppc_2025_tree_count_elig_round2'

# Today's date
today = datetime.today().strftime('%Y-%m-%d') # Check computer date before running (if out of sync, run sudo hwclock -s)

In [3]:
## FILES
# Input Files
# List of approved projects
approved_projects_file = '../projects_all_approved_202502211226.csv'

# PPC 2025 Batch 2 Projects
#ppc_batch2_file = '../data/ppc/ppc_2025_batch2_project_list.csv'

# PPC 2025 Potential Tree Count Projects (Eligibility Check #2)
ppc_tree_count_file = '../data/ppc_tree_count_elig/ppc_2025_potential_tree_count_projects_round2_2025-09-05.csv'

# Output Files
# JSON file to store the results of the TM API pull; read it back in to clean the results (outfile, infile)
tm_api_pull_results_file = f'../data/{run_dir}/tm_api_response_prod_{run_name}_{today}.json'

# CSV file to save the results of the TM API pull
polygon_features_file = f'../data/{run_dir}/tm_api_{run_name}_{today}.csv'

## Set up token & API URL

In [4]:
# Set up token access
auth_path = '../secrets.yaml'
with open(auth_path) as auth_file:
    auth = yaml.safe_load(auth_file)
headers = {
    'Authorization': f"Bearer {auth['access_token']}"
    }

In [5]:
# TerraMatch API URLs
staging_url = "https://api-staging.terramatch.org/research/v3/sitePolygons?" # Use for testing queries
prod_url = "https://api.terramatch.org/research/v3/sitePolygons?" # Use to pull data for analysis

## Create list of projects to pull

In [6]:
# Read in list of approved projects (2025-02-21)
full = pd.read_csv(approved_projects_file)

# Read in list of projects
tree_count_projects = pd.read_csv(ppc_tree_count_file)

In [ ]:
# Create lists of projects by Cohort (and split cohort 1 into projects within the TF landscapes and outside of the TF landscapes)
cohort1 = full[full['cohort'] == 'terrafund']
cohort1_landscapes = cohort1[cohort1['country'].isin(['BI', 'CD', 'RW', 'KE', 'GH'])]
cohort1_non_landscapes = cohort1[~cohort1['country'].isin(['BI', 'CD', 'RW', 'KE', 'GH'])]
cohort2 = full[full['cohort'] == 'terrafund-landscapes']

ppc = full[full['cohort'] == 'ppc']

In [ ]:
# Create a list of project ids to query
#ids = list(set(cohort1.project_id))
#len(ids)

# Create a short list of ids for testing 
#ids = ['244eaf7e-e109-47b2-b84e-9ebe24508391', '24d8c9a2-b8ef-481c-930b-78c9aeaf239e', 'f17dd6cf-8187-4edd-895e-07013d4990c9', '1115dda6-0165-4099-b52f-0ac53595c3a9']
#len(ids)

In [7]:
# Create a list of project IDs to query
## From the list of Tree Count Eligibility (Round 2) projects (9/5/25)
tree_count_proj_ids = list(tree_count_projects['project_id'].unique())
tree_count_proj_ids

['468bee12-bfbc-4387-a00a-d7e915576427',
 '02b3119e-9505-4dba-b58d-f2a967b71ef9',
 '5e8a3c5e-7a28-4ff4-be07-f950361f56b2',
 '1115dda6-0165-4099-b52f-0ac53595c3a9',
 '0dcfa582-bdc4-44dd-bb3d-b03a0fb5b67f']

## Pull projects from TerraMatch API

In [8]:
results = api.pull_wrapper(prod_url, headers, tree_count_proj_ids, outfile=tm_api_pull_results_file)

Pulling Projects: 100%|██████████| 5/5 [00:08<00:00,  1.73s/project]

Results saved to ../data/ppc_tree_count_elig/tm_api_response_prod_ppc_2025_tree_count_elig_round2_2025-09-05.json


## Parse and save the API output

In [9]:
# Load the saved JSON file
with open(tm_api_pull_results_file, 'r') as file:
    results = json.load(file)

In [10]:
# Convert the JSON output into a dataframe with selected fields
results_df = api.parse_tm_api_results(results, outfile = polygon_features_file, parse_indicators=True)

### Filter PPC polygons by desired plantstart years

In [11]:
# Read in polygons dataframe
df = pd.read_csv(polygon_features_file)

In [12]:
print(df.shape)
df.head()

(125, 17)


,project_id,project_name,poly_id,site_id,geometry,plantstart,plantend,practice,target_sys,dist,project_phase,tree_cover_loss,tree_cover_loss_fires,restoration_by_strategy,restoration_by_land_use,restoration_by_ecoregion,tree_cover
0,468bee12-bfbc-4387-a00a-d7e915576427,NaN,ce3557e6-f612-4966-8f1c-513a7fda54e4,dbfe61d7-1db5-45fc-9093-414a718c0b66,"{'type': 'Polygon', 'coordinates': [[[149.5189...",2022-09-15,NaN,tree-planting,NaN,NaN,NaN,"{'indicatorSlug': 'treeCoverLoss', 'yearOfAnal...","{'indicatorSlug': 'treeCoverLossFires', 'yearO...","{'indicatorSlug': 'restorationByStrategy', 'ye...","{'indicatorSlug': 'restorationByLandUse', 'yea...",NaN,NaN
1,468bee12-bfbc-4387-a00a-d7e915576427,NaN,43c9edda-3719-4e24-bdaf-0f4e43b55083,dbfe61d7-1db5-45fc-9093-414a718c0b66,"{'type': 'Polygon', 'coordinates': [[[149.5280...",2022-09-15,NaN,tree-planting,NaN,NaN,NaN,"{'indicatorSlug': 'treeCoverLoss', 'yearOfAnal...","{'indicatorSlug': 'treeCoverLossFires', 'yearO...","{'indicatorSlug': 'restorationByStrategy', 'ye...","{'indicatorSlug': 'restorationByLandUse', 'yea...",NaN,NaN
2,468bee12-bfbc-4387-a00a-d7e915576427,NaN,d92efb8f-359e-473f-aedf-e638d10364aa,7ab4843b-57e3-48fd-8cc8-05780312cf27,"{'type': 'Polygon', 'coordinates': [[[148.1825...",2022-09-15,NaN,"direct-seeding,tree-planting",NaN,NaN,NaN,"{'indicatorSlug': 'treeCoverLoss', 'yearOfAnal...","{'indicatorSlug': 'treeCoverLossFires', 'yearO...","{'indicatorSlug': 'restorationByStrategy', 'ye...","{'indicatorSlug': 'restorationByLandUse', 'yea...",NaN,NaN
3,468bee12-bfbc-4387-a00a-d7e915576427,NaN,9ccee886-5ece-4944-8e29-4293fecb44ee,8542c333-14b9-41a2-abe9-96adf9b91722,"{'type': 'Polygon', 'coordinates': [[[148.8751...",2022-09-15,NaN,tree-planting,NaN,NaN,NaN,"{'indicatorSlug': 'treeCoverLoss', 'yearOfAnal...","{'indicatorSlug': 'treeCoverLossFires', 'yearO...","{'indicatorSlug': 'restorationByStrategy', 'ye...","{'indicatorSlug': 'restorationByLandUse', 'yea...",NaN,NaN
4,468bee12-bfbc-4387-a00a-d7e915576427,NaN,645b3d19-76f3-461f-b6da-a35eb8a9cd15,ff2b41f4-cb46-4126-85f7-fad687435db6,"{'type': 'Polygon', 'coordinates': [[[149.1120...",2022-09-15,NaN,tree-planting,NaN,NaN,NaN,"{'indicatorSlug': 'treeCoverLoss', 'yearOfAnal...","{'indicatorSlug': 'treeCoverLossFires', 'yearO...","{'indicatorSlug': 'restorationByStrategy', 'ye...","{'indicatorSlug': 'restorationByLandUse', 'yea...",NaN,NaN


In [13]:
# Filter the polygons dataframe by the year(s) of interest
results_df_filt = clean.filter_by_years_of_interest(polygons_df=df, years_df=tree_count_projects)

In [14]:
print(f"results_df had {len(results_df)} polygons")
print(f"results_df_filt has {len(results_df_filt)} polygons")

results_df had 125 polygons
results_df_filt has 122 polygons


In [15]:
# Save the filtered dataframe (overwrite the unfiltered data)
results_df_filt.to_csv(f"../data/{run_dir}/tm_api_{run_name}_{today}.csv", index=False)